# Extrinsic Curvature Computations 

### NGSolve User Meeting, June 2025, Hamburg

$
\newcommand{\bb}[1]{\mathbb{{#1}}}
\newcommand{\cl}[1]{\mathcal{{#1}}}
\newcommand{\PS}{P_{{\cl{S}}}}
\newcommand{\gradS}{\nabla_{\cl{S}}}
$

In [ ]:
from netgen.occ import X, Y, Z, Pnt, Sphere, Cylinder, Ellipsoid, Axes, OCCGeometry
from netgen import meshing
from ngsolve import Mesh, Cof, Grad, Integrate, Trace, Cross, Det, OuterProduct, InnerProduct, Normalize
from ngsolve import tan, cosh, cos, acos, sin, ds, log, sqrt, pi
from ngsolve import VectorFacetSurface, HDivDivSurface, LinearForm, BilinearForm, GridFunction
import ngsolve as ng
from ngsolve.webgui import Draw
MESHSURF = meshing.MeshingStep.MESHSURFACE
curving_order = 3

## Curvature of embedded smooth manifolds

In the extrinsic approach, we consider an orientable surface $\cl{S}$, smoothly embedded in the Euclidean space $\bb{R}^3$, possessing a smooth unit normal field $\nu$. It is very intuitive to measure the curvature of $\cl S$ by examining how the normal $\nu$ changes. The standard extrinsic approach proceeds to define curvature  using the derivative of $\nu$ on the surface.

One approach to differentiating fields on $\cl S$ is by using standard partial derivatives of their smooth extensions in $\bb R^3$ (from $\cl S$). We need the orthogonal (Euclidean) projection onto the tangent space of the surface, represented by the action of the matrix  

$$
\PS = I - \nu \nu^T.
$$

Letting $\nabla$ denote the usual gradient in three dimensions, we define the surface gradients of a scalar field $f$ and a vector field $u = (u_1, u_2, u_3)$ on the surface  $\cl{S}$ by $\nabla_{\mathcal{S}} f := \PS \nabla f$ and 

$$
\gradS u := (\PS \nabla u_1, \PS \nabla u_2, \PS \nabla u_3),
$$

respectively. Clearly the surface gradient of a vector field is a $3 \times 3$ matrix field.

This surface gradient $\nabla_{\cl S}$ can be computed in NGSolve using `Grad`. NGSolve does not compute it using extensions as mentioned above, but rather by mappings from a reference element and a pseudoinverse, an alternate technique to compute the same surface gradient.

The **Weingarten tensor**, or the matrix of the *shape operator* is the $3\times 3$ matrix field defined by 

$$
W =\gradS \nu.
$$

Extrinsic curvature definitions are based on $W$. Recall the following:

1.  $W$ is a symmetric matrix at every point of $\cl S$.
2.  $W$ always has zero as one eigenvalue with $\nu$ as the corresponding eigenvector.
3.  Its other two eigenvalues $k_1$ and $k_2$ are called the two **principal curvatures** of the surface $\cl S$.
4.  The **Gauss curvature** $K$ and the **mean curvature** $H$  of $\cl S$ are given, respectively, by

\begin{align*}
K &=  k_1 \, k_2, \\
H &= \frac 1 2 (k_1 + k_2).
\end{align*}

We now show how to compute these curvatures in NGSolve.

## Example 1. Curvature of a sphere

A sphere curves in exactly the same way in all directions, so all its principal curvatures must be equal. They equal the reciprocal of its radius $r$. It offers an example where $W$ and all the above-mentioned curvatures can be analytically computed:
\begin{align*}
k_i = \frac{1}{r},\qquad K=\frac{1}{r^2},\qquad H=\frac{1}{r}.
\end{align*}
The sphere is called an *elliptic surface* as its Gauss curvature is positive.

We make a sphere of radius `r` in OCC geometry and mesh it using `netgen` as follows. Note that `curving_order` determines how accurately the computational object represents the geometry through mappings from a reference element.

In [ ]:
spheremesh = Mesh(OCCGeometry(Sphere(Pnt(0,0,0), r=2)).GenerateMesh(maxh=0.4, perfstepsend=MESHSURF))
spheremesh.Curve(curving_order);
Draw(spheremesh);

The unit normal vector $\nu$ of the sphere and the Weingarten tensor can now be computed easily in ngsolve. 

In [ ]:
nu = ng.specialcf.normal(3)  
W = Grad(nu)   # Weingarten tensor

The `dims` attribute of an ngsolve `CoefficientFunction` verifies that `W` is a $3\times 3$ matrix field.

In [ ]:
print(W.dims)

As mentioned previously, `W` has a zero eigenvalue with the normal $\nu$ as the corresponding eigenvector. Since the other two eigenvalues are the principal curvatures, $\kappa_i$, their sum equals the trace of `W`. Hence the mean curvature $H$ can be computed using the matrix `Trace`:

In [ ]:
H = 0.5*Trace(Grad(nu))
Draw (H, spheremesh, 'Mean curvature'); 

Clearly the computed mean curvature agrees with the exact mean curvature up to what appears to be numerical errors. By increasing the `curving_order`  and repeating the above computations, you will see these errors reduce.

To compute the Gauss curvature $K = \kappa_1 \kappa_2$ from the singular matrix `W`, one can compute all  eigenvalues of $W$, then isolate the nonzero ones, and take their product. Alternately, one can change $W$'s zero eigenvalue to one and then compute the product of all eigenvalues through a  determinant computation, as done next. The matrix 

$$
\tilde W = W + \nu \nu^t
$$

continues to have $\nu$ as an eigenvector but with eigenvalue one. The remainder of its spectrum coincides with that of $W$. Hence

$$
\det \tilde W =  k_1 k_2 = K.
$$


In [ ]:
K = Det(W + OuterProduct(nu, nu))
Draw (K, spheremesh, 'Gauss curvature'); 

Once we have $H$ and $K$, we can simply solve a quadratic equation to get the principal curvatures $k_1, k_2$. For repeated use, we collect these formulae into a python function next.

In [ ]:
def ExtrinsicCurvatureSmooth(mesh):
    """    H, K, k1, k2 = ExtrinsicCurvatureSmooth(mesh)
    Return mean curvature H, Gauss curvature K, and principal curvatures k1, k2. 
    """
    nu = ng.specialcf.normal(3)
    W = Grad(nu)
    H = 0.5 * Trace(W)
    K = Det(W + OuterProduct(nu, nu))
    k1 = H + sqrt(H**2 - K)
    k2 = H - sqrt(H**2 - K)
    return H, K, k1, k2

## Example 2. Curvature of a cylinder

Let us make a cylinder along the `X` axis of radius `r` and length `h`.

In [ ]:
cylmesh = Mesh(OCCGeometry(Cylinder((0,0,0), X, r=2, h=3).faces[0]).GenerateMesh(maxh=0.5)).Curve(curving_order)
Draw(cylmesh);

A cylinder curves in just one direction. Hence we expect one of its principal curvatures to equal zero, and the other equal to the reciprocal of the radius $r$. One can analytically compute
\begin{align*}
\kappa_1 =0, \quad \kappa_2 = \frac{1}{r},\qquad K=0,\qquad H=\frac{1}{2r}.
\end{align*}
The cylinder is a surface of *parabolic type* as its Gauss curvature is zero.

In [ ]:
H, K, k1, k2 = ExtrinsicCurvatureSmooth(cylmesh)
Draw(k1, cylmesh, 'k1'); Draw(k2, cylmesh, 'k2');

### Example 3: Curvature of a pseudosphere

A *hyperbolic surface* has negative Gauss curvature, i.e., its  principal curvatures have opposite signs. A pseudosphere (also known as a [tractroid](https://en.wikipedia.org/wiki/Pseudosphere) since it is the revolution of a tractrix) of negative Gauss curvature

$$
K = -1
$$

is given as follows.

In [ ]:
def gu(u):
    return log(tan(u/2)) + cos(u)

def mapping(x, y, z):
    return (sin(pi/9/2+x*pi/9)*cos(y*2*pi),
            sin(pi/9/2+x*pi/9)*sin(y*2*pi),
            gu(pi/9/2+x*pi/9) )

psphmesh = Mesh(meshing.SurfaceGeometry(mapping).GenerateMesh(quads=False,nx=8,ny=16)).Curve(curving_order)
Draw(psphmesh)

In [ ]:
H, K, k1, k2 = ExtrinsicCurvatureSmooth(psphmesh)
print('Gauss curvature:')
Draw(K, psphmesh, 'K');  
print('Principal curvatures:')
Draw(k1, psphmesh, 'k1'); Draw(k2, psphmesh, 'k2');

## Nonsmooth surfaces

When the surface is not smooth, jumps of its normal vector $\nu$ creates "distributional" curvature along the edges where $\nu$ is discontinuous. Additional terms along such edges are needed to  provide a notion of shape operator for piecewise smooth surfaces. This notion, while important on its own, is also useful in numerical computations that proceed by approximating a smooth surface by a piecewise flat surface. 


Suppose $\cl S$ is a piecewise smooth surface that is locally continuous (but not necessarily $C^1$) and is partitioned into a triangulation $\cl T$. Each element $T$ of the triangulation $\cl T$ is a smooth surface with a smooth normal $\nu_T$. But the normal need not be smooth across an edge $E$ shared by two elements $T_\pm$. Along such an edge, the unit normals from either side make an angle
$$
\sphericalangle\nu := \cos^{-1}(\nu_{T_+}, \nu_{T_-}).
$$
The jump of the normal as measured by this angle is a source of curvature in the generalization that follows.

We define the **generalized Weingarten tensor** of the piecewise smooth surface $\cl S$, denoted by $\widetilde{\nabla_{\cl S} \nu}$, as a functional acting on a symmetric matrix fields $\sigma$, as follows:
$$
\widetilde{\nabla_{\cl S} \nu} (\sigma)
= \sum_{T \in \mathcal{T}}\int_T\nabla\nu_T:{\sigma}\,ds 
+ \sum_{E\in\mathcal{E}}\int_{E} \sphericalangle\nu \; \mu \mu^t : {\sigma}\,dl
$$
Here $\mu = \nu \times t$  is a co-normal vector of along an edge $E$  with unit tangent vector $t$,
and  $\sigma$ is taken to be any function in the surface Hellan-Herrmann-Johnson (HHJ) finite element space $M_h$ of order $k-1$ if $k$ is the polynomial order of curving. Such a $\sigma$ has continuous 
$ \sigma_{\mu\mu} = (\sigma \mu )\cdot \mu$
across the edges even when $\mu$ and $\sigma$ are discontinuous. The sum over edges in the above definition can be rewritten as a sum over element boundaries after introducing an  intermediary averaged normal vector
$\{\nu\} = (\nu_{T_+} +\nu_{T_-} ) / (\|\nu_{T_+} + \nu_{T_-}\|) $ and using the angle between  the outward unit co-normal $\mu$ on and $\{\nu\}$, denoted by $\sphericalangle^\mu_{\{ \nu\}}$, as follows:
$$
\widetilde{\nabla_{\cl S} \nu} (\sigma)
= \sum_{T \in \mathcal{T}}\left( \int_T\nabla\nu_T:{\sigma}\,ds 
+ \int_{\partial T} \left(\frac{\pi}{2}-\sphericalangle^\mu_{\{ \nu\}} \right)
{\sigma}_{\mu\mu}\,dl \right)
$$
We will compute with this  expression as it is numerically more stable when  $\mu\cdot\{\nu\} \approx 0$, an often occurring case when the $\cl{T}$ approaches a smooth surface $\cl S$.


One final issue with such distribution-like functionals is that they cannot be immediately visualized like functions. We must lift the functional to a function-representation which can be visualized as usual in NGSolve. Therefore we compute  the *lifted generalized Weingarten tensor* $\widetilde W$ as the unique element in $M_h$ such that 
\begin{align*}
\int_S\widetilde W: {\sigma}\,ds = \widetilde{\nabla_{\cl S} \nu} (\sigma)
\end{align*}
holds for all $\sigma$ in $M_h$. The visualizations below are those of $\widetilde W$. 

We begin by implementing the above formula for $\widetilde{\nabla_{\cl S} \nu}$ as a `LinearForm`-functional on the surface HHJ space, called `HDivDivSurface` in ngsolve. Then, we invert a mass matrix in the HHJ space to  compute the lifting $\widetilde W$.

In [ ]:
def GeneralizedWeingarten(mesh, lifting_order=curving_order-1):

    # Orthonormal (nu, t, mu) system (used on element boundaries)
    nu = ng.specialcf.normal(3)
    t = ng.specialcf.tangential(3)
    mu = Cross(nu, t)

    # HHJ space on surface: 
    liftWspace = HDivDivSurface(mesh, order=lifting_order)
    sigma, tau = liftWspace.TnT()
    sigma, tau = sigma.Trace(), tau.Trace()

    # Average normal vector nu_avg = (nu_+  + nu_-)/2
    nu_avg = GridFunction(ng.VectorFacetSurface(mesh, order=lifting_order))
    nu_avg.Set(nu, dual=True, definedon=mesh.Boundaries(".*"))

    # Sources for generalized Weingarten tensor
    f = LinearForm(liftWspace)
    f += InnerProduct(Grad(nu), tau) * ds
    f += (pi / 2 - acos(Normalize(nu_avg) * mu)) * tau[mu, mu]  \
        * ds(element_boundary=True)

    # Lift distributional W into HDivDivSurface space
    a = BilinearForm(InnerProduct(sigma, tau) * ds, symmetric=True)
    liftW = GridFunction(liftWspace)
    with ng.TaskManager():
        a.Assemble()
        f.Assemble()
        liftW.vec.data =  \
            a.mat.Inverse(liftWspace.FreeDofs(),
                          inverse="sparsecholesky") * f.vec

    return liftW

As an example, we consider a smooth surface, an ellipse, whose smooth mean curvature looks like this:

In [ ]:
ell = Ellipsoid(Axes((0, 0, 0), X, Y), r1=1, r2=1.5, r3=0.8).faces[0]

# Draw the smooth Weingarten tensor for viewing the exact baseline:
ellipsemesh = Mesh(OCCGeometry(ell).GenerateMesh(maxh=1)).Curve(5)
Draw(ExtrinsicCurvatureSmooth(ellipsemesh)[0], ellipsemesh);

Now we approximate the smooth ellipse by a *piecewise flat* surface. 

In [ ]:
Draw(Mesh(OCCGeometry(ell).GenerateMesh(maxh=0.5)).Curve(1));

If we only account for the Weingarten tensor within elements, then we obtain zero curvatures! Clearly, for piecewise flat manifolds, all the curvature information resides in the edge contributions.  So we use the above-mentioned (lifted) generalized Weingarten tensor to compute an approximation to the mean curvature.

In [ ]:
def DrawMeanCurvatureApprox(maxh=0.5, order=1):
    ellmesh = Mesh(OCCGeometry(ell).GenerateMesh(maxh=maxh)).Curve(order)
    W = GeneralizedWeingarten(ellmesh, lifting_order=order-1)
    Draw(0.5 * Trace(W), ellmesh);
    return W

In [ ]:
DrawMeanCurvatureApprox(maxh=0.5);

On a finer mesh, the approximation improves.

In [ ]:
DrawMeanCurvatureApprox(maxh=0.1);

Clearly, as the mesh size gets finer, the approximation appears to converge to the smooth mean curvature of the original smooth ellipse. 